In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json 
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout


# from keras.layers import Dense, 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [3]:
with open('/content/drive/MyDrive/Files/GL Bot.json') as f:
  data = pd.read_json(f)

In [4]:
data['intents'][0]

{'context_set': '',
 'patterns': ['hi',
  'how are you',
  'is anyone there',
  'hello',
  'whats up',
  'hey',
  'yo',
  'listen',
  'please help me',
  'i am learner from',
  'i belong to',
  'aiml batch',
  'aifl batch',
  'i am from',
  'my pm is',
  'blended',
  'online',
  'i am from',
  'hey ya',
  'talking to you for first time'],
 'responses': ['Hello! how can i help you ?'],
 'tag': 'Intro'}

In [5]:
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import random

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
labels = []
words=[]
documents = []
ignore_words = ['?', '!']

for intent in data['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in labels:  
      labels.append(intent['tag'])     
num_classes = len(labels)

In [8]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
labels = sorted(list(set(labels)))

In [9]:
# initializing training data
sentences = []
output_empty = [0] * len(labels)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[labels.index(doc[1])] = 1

    sentences.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(sentences)
sentences = np.array(sentences)
# create train and test lists. X - patterns, Y - intents
X = list(sentences[:,0])
y = list(sentences[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
model = Sequential()
model.add(Dense(50, input_shape=(len(X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                7950      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
Total params: 9,174
Trainable params: 9,174
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(np.array(X), np.array(y), epochs=100, batch_size = 4, verbose = 1)

Epoch 1/100
32/32 [==============================] - 1s 2ms/step - loss: 2.0672 - accuracy: 0.1484
Epoch 2/100
32/32 [==============================] - 0s 1ms/step - loss: 2.0423 - accuracy: 0.2422
Epoch 3/100
32/32 [==============================] - 0s 1ms/step - loss: 2.0067 - accuracy: 0.3359
Epoch 4/100
32/32 [==============================] - 0s 1ms/step - loss: 1.9578 - accuracy: 0.2891
Epoch 5/100
32/32 [==============================] - 0s 1ms/step - loss: 1.9153 - accuracy: 0.3203
Epoch 6/100
32/32 [==============================] - 0s 2ms/step - loss: 1.8576 - accuracy: 0.2344
Epoch 7/100
32/32 [==============================] - 0s 1ms/step - loss: 1.7750 - accuracy: 0.2578
Epoch 8/100
32/32 [==============================] - 0s 1ms/step - loss: 1.7072 - accuracy: 0.3203
Epoch 9/100
32/32 [==============================] - 0s 2ms/step - loss: 1.6396 - accuracy: 0.3828
Epoch 10/100
32/32 [==============================] - 0s 2ms/step - loss: 1.5695 - accuracy: 0.3984
Epoch 11/

In [12]:
# to save the trained model
model.save("/content/drive/MyDrive/Files/chat_model")

import pickle

# to save the fitted tokenizer
with open('words.pickle', 'wb') as handle:
    pickle.dump(words, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('labels.pickle', 'wb') as ecn_file:
    pickle.dump(labels, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Files/chat_model/assets


In [13]:
import json 
import numpy as np
from tensorflow import keras
import random
import pickle

with open('/content/drive/MyDrive/Files/GL Bot.json') as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('/content/drive/MyDrive/Files/chat_model')
    ignore_words = ['?', '!']
    # load tokenizer object
    with open('words.pickle', 'rb') as handle:
        words = pickle.load(handle)

    # load label encoder object
    with open('labels.pickle', 'rb') as enc:
        labels = pickle.load(enc)

    print("\nChatBot: Hey There! How may I help you?")

    while True:
        
        print("\nUser: ")
        inp = input()
        if inp.lower() == "quit":
            break

        sentence_words = nltk.word_tokenize(inp)
        sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

        bag = [0]*len(words)
        for s in sentence_words:
            for i,w in enumerate(words):
                if w == s:
                    bag[i] = 1
        bag =  np.array(bag)
      
        result = model.predict(np.array([bag]))[0]
        ERROR_THRESHOLD = 0.25
        res = [[i,r] for i,r in enumerate(result) if r>ERROR_THRESHOLD]
        # sort by strength of probability
        res.sort(key=lambda x: x[1], reverse=True)
        output_list = []
        for r in res:
          output_list.append({"intent": labels[r[0]], "probability": str(r[1])})

        tag = output_list[0]['intent']
        list_of_intents = data['intents']
        for i in list_of_intents:
          if(i['tag'] == tag):
            result = random.choice(i['responses'])
            print("ChatBot:", np.random.choice(i['responses']))
print("Start messaging with the bot (type quit to stop)!")
chat()

Start messaging with the bot (type quit to stop)!

ChatBot: Hey There! How may I help you?

User: 
Hi!!!
ChatBot: Hello! how can i help you ?

User: 
Neural Networks
ChatBot: Link: Neural Nets wiki

User: 
Supervised Learning
ChatBot: Link: Machine Learning wiki 

User: 
Support Request
ChatBot: Link: Machine Learning wiki 

User: 
Olympus
ChatBot: Link: Olympus wiki

User: 
Thank You
ChatBot: I hope I was able to assist you, Good Bye

User: 
Quit
